# Listed CDS Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-cds-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [1]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
# IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DEKSTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.edp.thomsonreuters.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### retrieving proceeds and sensitivities for a Listed CDS

This sample computes the following CDS analytics :
- the option term & conditions (InstrumentDescription,EndDate,DealSpreadBp)
- the proceeds (MarketValueInDealCcy,AccruedAmountInDealCcy,UpfrontAmountInDealCcy)
- the main sensitivities (ConventionalSpreadBp,ParSpreadBp,CR01AmountInDealCcy, DV01AmountInDealCcy,RR01AmountInDealCcy,JumpToDefaultAmountInDealCcy)
- the error information (ErrorCode,ErrorMessage)

the full documentation of the parameters available for equity options can be found in the [Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-cds-contracts)


In [2]:
request_body = {
    
    "fields": ["InstrumentDescription","EndDate","DealSpreadBp","CreditCurveId",
               "MarketValueInDealCcy","AccruedAmountInDealCcy","UpfrontAmountInDealCcy",
               "ConventionalSpreadBp","ParSpreadBp","CR01AmountInDealCcy","DV01AmountInDealCcy","RR01AmountInDealCcy","JumpToDefaultAmountInDealCcy",
               "ErrorCode","ErrorMessage"],
    
    "universe":[{
        "instrumentType": "CDS",
        "instrumentDefinition": {
            "cdsConvention":"ISDA",
            "instrumentCode": "FRGV5YUSAC=R"
        }
    }],     
    
    "pricingParameters": {
        "valuationDate": "2021-01-12T00:00:00Z"
    },
        
    "outputs" : ["Data","Headers"],
}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentDescription'}, {'type': 'DateTime', 'name': 'EndDate'}, {'type': 'Float', 'name': 'DealSpreadBp'}, {'type': 'String', 'name': 'CreditCurveId'}, {'type': 'Float', 'name': 'MarketValueInDealCcy'}, {'type': 'Float', 'name': 'AccruedAmountInDealCcy'}, {'type': 'Float', 'name': 'UpfrontAmountInDealCcy'}, {'type': 'Float', 'name': 'ConventionalSpreadBp'}, {'type': 'Float', 'name': 'ParSpreadBp'}, {'type': 'Float', 'name': 'CR01AmountInDealCcy'}, {'type': 'Float', 'name': 'DV01AmountInDealCcy'}, {'type': 'Float', 'name': 'RR01AmountInDealCcy'}, {'type': 'Float', 'name': 'JumpToDefaultAmountInDealCcy'}, {'type': 'String', 'name': 'ErrorCode'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [['FRANCE.USD.SNRFOR.CR', '2025-12-20T00:00:00Z', 100.0, 'CdsIssuerCurve0#FRGVUSACRBMK=2021-01-11T00:00:00', 41841.9299827525, 638.888888888889, -41203.1595873435, 16.3789, 16.556908305022, 510.120923565904, 10.5709936042549, 137.014991321666, 661203.1

In [3]:
import pandas as pd

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentDescription,EndDate,DealSpreadBp,CreditCurveId,MarketValueInDealCcy,AccruedAmountInDealCcy,UpfrontAmountInDealCcy,ConventionalSpreadBp,ParSpreadBp,CR01AmountInDealCcy,DV01AmountInDealCcy,RR01AmountInDealCcy,JumpToDefaultAmountInDealCcy,ErrorCode,ErrorMessage
0,FRANCE.USD.SNRFOR.CR,2025-12-20T00:00:00Z,100.0,CdsIssuerCurve0#FRGVUSACRBMK=2021-01-11T00:00:00,41841.929983,638.888889,-41203.159587,16.3789,16.556908,510.120924,10.570994,137.014991,661203.159587,,
